In [9]:
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
master_index_all_states_df = pd.read_csv('/Users/Documents/Research/SERFF/master_index_all_states.csv')

In [11]:
master_index_all_states_df.head(5)

,Company Name,NAIC Company Code,Insurance Product Name,Sub Type of Insurance,Filing Type,Filing Status,SERFF Tracking Number,State
0,XL Specialty Insurance Company,37885.0,Excess Workers Compensation,16.0003 Excess WC,Form,Closed - Approved,XLAM-129754727,VT
1,XL Specialty Insurance Company,37885.0,Excess Workers Compensation,16.0003 Excess WC,Form,Closed - Approved,XLAM-132249557,VT
2,XL Specialty Insurance Company,37885.0,Excess Workers Compensation,16.0003 Excess WC,Form,Closed - Approved,XLAM-129498898,VT
3,XL Specialty Insurance Company,37885.0,Fetch Pet Health Insurance Program 2023 Revisi...,09.0004 Pet Insurance Plans,Form/Rate/Rule,Closed - Approved,XLAM-133698612,VT
4,XL Specialty Insurance Company,37885.0,Fetch Pet Health Insurance Program 2023 Revisi...,09.0004 Pet Insurance Plans,Rate/Rule,Closed - Rate/Rule-P&C Use and File,XLAM-133863357,VT


# Feature Transformation

## Construct Filing Type Indicator

### Check Model Related Filing Type

In [17]:
contains_model = master_index_all_states_df[['Filing Type']].applymap(lambda x: 'model' in str(x).lower())

# Sum the boolean DataFrame to count True values (i.e., where 'model' appears)
row_count_with_model = contains_model.any(axis=1).sum()

In [18]:
row_count_with_model # only 815 filings contain the word model

815

In [19]:
master_index_all_states_df['Filing Type - Form'] = ['Y' if 'form' in str(x).lower() else 'N' for x in master_index_all_states_df['Filing Type']]

In [20]:
rate_related_keywords = ['rate','loss cost','lcm','rating','model']


In [21]:
master_index_all_states_df['Filing Type - Rate'] = ['Y' if any([y in str(x).lower() for y in rate_related_keywords]) else 'N' for x in master_index_all_states_df['Filing Type']]


In [22]:
master_index_all_states_df[master_index_all_states_df['Filing Type - Rate']=='Y'].shape

(1073306, 10)

In [23]:
master_index_all_states_df[master_index_all_states_df['Filing Type - Form']=='Y'].shape


(1915313, 10)

## Type of Insurance

### Select Auto

In [24]:
master_index_all_states_df['Type of Insurance'] = [str(x)[:4] for x in master_index_all_states_df['Sub Type of Insurance']]


In [25]:
def can_convert_to_float(s):
    try:
        float(s)  # Try converting the string to float
        return True
    except ValueError:  # Catch the exception if conversion fails
        return False

In [26]:
master_index_all_states_df['Type of Insurance - Indicator'] = [can_convert_to_float(x) for x in master_index_all_states_df['Type of Insurance']]


In [28]:
master_index_all_states_df['Type of Insurance - Indicator'].sum()/master_index_all_states_df.shape[0]

0.9620996993325793

In [38]:
def identify_personal_auto_filing(type_of_insurance):
    type_of_insurance = str(type_of_insurance).lower()
    if '19.0' in type_of_insurance:
        return 'Y'
    elif 'auto' in type_of_insurance and 'personal' in type_of_insurance:
        return 'Y'
    elif 'auto' in type_of_insurance and 'private' in type_of_insurance:
        return 'Y'
    else:
        return 'N'

In [39]:
master_index_all_states_df['Type of Insurance - Auto'] = [identify_personal_auto_filing(x) for x in master_index_all_states_df['Type of Insurance']]

In [40]:
master_index_all_states_df[master_index_all_states_df['Type of Insurance - Auto']=='Y'].shape # 258244

(258244, 13)

In [41]:
master_index_all_states_df[master_index_all_states_df['Type of Insurance - Auto']=='Y']['State'].nunique()

48

# State Selection

In [7]:
# Identify the states with the most number of filings
state_counts = master_index_all_states_df['State'].value_counts()

# Get the top 5 states with the most rows
top_5_states = state_counts.head(10)

# Display the result
print(top_5_states)

WI    115707
MD    111478
GA    111045
OK    109999
MO    103448
PA    100622
IL     95542
CT     95034
NC     90834
IN     84536
Name: State, dtype: int64


In [8]:
state_naic_counts = master_index_all_states_df.groupby(['State'])['NAIC Company Code'].nunique()

# Summarize to get the total number of unique companies per state
state_company_counts = state_naic_counts.groupby(level='State').sum()

# Sort the counts in descending order and get the top states
top_states_by_companies = state_company_counts.sort_values(ascending=False).head(10)

# Display the results
print(top_states_by_companies)

State
GA    1202
MO    1109
PA    1059
IN    1017
TN     991
OK     986
NM     981
IL     978
MD     969
WI     963
Name: NAIC Company Code, dtype: int64


# Select PA, Auto and Rate

In [42]:
index_PA_Auto_Rate_df = master_index_all_states_df[(master_index_all_states_df['State']=='PA')&
                                                (master_index_all_states_df['Type of Insurance - Auto']=='Y')&
                                                (master_index_all_states_df['Filing Type - Rate']=='Y')]

In [43]:
index_PA_Auto_Rate_df.to_csv('/Users/qiaojiang/Documents/Research/SERFF/index_PA_Auto_Rate_df.csv', index = False)

In [44]:
index_PA_Auto_Rate_df.shape

(4326, 13)

# Select PA, Auto and Form

In [45]:
index_PA_Auto_Form_df = master_index_all_states_df[(master_index_all_states_df['State']=='PA')&
                                                (master_index_all_states_df['Type of Insurance - Auto']=='Y')&
                                                (master_index_all_states_df['Filing Type - Form']=='Y')]

In [46]:
index_PA_Auto_Form_df.to_csv('/Users/Documents/Research/SERFF/index_PA_Auto_Form_df.csv', index = False)

In [47]:
index_PA_Auto_Form_df.shape

(3083, 13)